In [2]:
from robot_menagerie import xarm7_description, xarm7_mj_description
# from mujoco_simulator.model_builder import ModelBuilder
# from mujoco_simulator.simulator import MujocoRender
import numpy as np
from darli.model import Model, Functional
from darli.backend import CasadiBackend
from darli.backend.liecasadi.so3 import SO3,SO3Tangent
import casadi as cs


ModuleNotFoundError: No module named 'robot_menagerie'

In [2]:
model_wrapper = ModelBuilder(xarm7_mj_description.MJCF_PATH)
model_wrapper.add_arena()
model_wrapper.build()
ren = MujocoRender(model=model_wrapper, 
                   start=True)

Process render is starting!


In [3]:
model = Functional(CasadiBackend(xarm7_description.URDF_PATH))
model.add_body({'ee': "link7"})

In [4]:
opti_problem = cs.Opti()
q = opti_problem.variable(model.nq)

In [5]:
ee_pos = model.body('ee').position(q)
ee_xyzw = model.body('ee').quaternion(q)

In [6]:
# TODO: move to parameters
desired_pos = opti_problem.parameter(3)
desired_xyzw = opti_problem.parameter(4)

In [7]:
so3_error = SO3(xyzw = ee_xyzw).distance(SO3(xyzw = desired_xyzw))**2 #SO3.from_matrix(ee_rot) - SO3.from_matrix(desired_rot)  

pos_cost = cs.sumsqr(ee_pos - desired_pos)
so3_cost = 0.5*so3_error#cs.sumsqr(so3_error.vec)

In [8]:
cost = pos_cost + so3_cost
opti_problem.minimize(cost)
# opti_problem.subject_to(pos_cost == 0)
opti_problem.subject_to(opti_problem.bounded(model.q_min, q, model.q_max))
opti_problem.solver('ipopt')

In [9]:
pos_des = [0.4, 0, 0.4]
xyzw_des = [1, 0,  0, 0]

In [10]:

opti_problem.set_value(desired_pos,pos_des)
q0_random = np.clip(np.random.randn(model.nq), model.q_min, model.q_max)
opti_problem.set_initial(q, q0_random)
opti_problem.set_value(desired_xyzw, xyzw_des)

In [11]:
opti_problem.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        7
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

OptiSol(Opti {
  instance #0
  #variables: 1 (nx = 7)
  #parameters: 2 (np = 7)
  #constraints: 1 (ng = 7)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [12]:
q_opt = opti_problem.value(q)
q_opt

array([-0.38808609, -0.26470144,  0.35456735,  0.91028185,  0.09906989,
        1.16041696, -0.08462904])

In [13]:
np.array(model.body('ee').position(q_opt))

array([[ 3.99999999e-01],
       [-4.13486624e-09],
       [ 3.99999995e-01]])

In [14]:
pos_des

[0.4, 0, 0.4]

In [15]:
xyzw_opt = model.body('ee').quaternion(q_opt)
xyzw_opt

DM([1, 7.59084e-05, 0.000292654, 7.981e-05])

In [16]:
xyzw_des

[1, 0, 0, 0]

## Iterative solving with Regulization

In [17]:

previous_configuration = opti_problem.parameter(model.nq)
weights = opti_problem.parameter(3)
delta = opti_problem.parameter(1)
v_max = opti_problem.parameter(1)
T = opti_problem.parameter(1)
v = (q - previous_configuration)/T
opti_problem.subject_to(opti_problem.bounded(-v_max, v, v_max))

def pseudo_huber_loss(x, delta = 0.2):
    return delta**2 * (cs.sqrt(1 + (x / delta)**2) - 1)

# cost = weights[0]*pos_cost + weights[1]*so3_cost + weights[2]*cs.sumsqr(previous_configuration - q)
cost = weights[0]*cs.sum1(pseudo_huber_loss(ee_pos - desired_pos, delta))+ weights[1]*so3_cost + weights[2]*cs.sumsqr(previous_configuration - q)

opti_problem.minimize(cost)
opti_problem.solver(
            "ipopt",
            {
                "ipopt.print_level": 0,
                'ipopt.max_iter': 5,
                "print_time": 0,
                "ipopt.warm_start_init_point": "yes"
            },
        )


In [18]:

# opti_problem.solver('sqpmethod')
from time import perf_counter, sleep
q0 = np.array([0,0,0,np.pi/2,0,np.pi/2, 0])
opti_problem.set_value(previous_configuration,q0)
opti_problem.set_initial(q, q0)
opti_problem.set_value(weights[0], 0.5)
opti_problem.set_value(weights[1], 2)
opti_problem.set_value(weights[2], 0.001)
opti_problem.set_value(delta, 0.5)
opti_problem.set_value(v_max, 5.)
opti_problem.set_value(T, 0.005)


xyzw_des = [1,0,0,0]

for j in range(3):
    rand_point = 0.1*np.random.rand(3)
    for i in range(700):
        
        if i <= 200:
            opti_problem.set_value(weights[2], 0.4)
            pos_des = [0.4, 0, 0.4] + rand_point
        if i>200:
            pos_des = [0.4, 0, 0.4] + rand_point
            pos_des[j] = pos_des[j] + 0.15*np.sin(2*np.pi*i/200)
            opti_problem.set_value(weights[2], 0.001)
        if i >= 500:
            opti_problem.set_value(weights[2], 0.4)
            pos_des = [0.4, -0.1, 0.5] + rand_point
            
        opti_problem.set_initial(q, q_opt)
        opti_problem.set_value(desired_pos,pos_des)
        opti_problem.set_value(desired_xyzw,xyzw_des)
        opti_problem.set_value(previous_configuration,q_opt)
        
        t1 = perf_counter()
        
        # Solve the optimization problem
        try:
            sol = opti_problem.solve()
            q_opt = opti_problem.value(q)
            v_opt = opti_problem.value(v)
        except Exception as e:
            q_opt = opti_problem.debug.value(q)
            v_opt = opti_problem.debug.value(v)
        
        t2 = perf_counter()
        
        pos_act = np.array(model.body('ee').position(q_opt)).reshape(3)
        print(1000*(t2 - t1))
        
        # RENDER:
        ren.set_state(q_opt)
        
        ren.markers[0](position=pos_act.copy(),
                            color=[1, 0, 0, 0.5],
                            size=0.02)
        
        ren.markers[1](position=np.array(pos_des).copy(),
                            color=[0, 0, 1, 0.5],
                            size=0.022)

        sleep(0.005)

7.341433000000563
3.728332000001444
2.699739000000534
3.02487999999812
2.6993370000027994
3.0696820000031266
2.730597000002888
2.879513999999972
2.777595000004851
2.8524210000000494
2.624797000002843
2.8479069999960416
7.181295999998838
2.5506760000055806
3.2477249999942615
5.723011999997141
2.4695060000041735
2.3360499999967033
2.7878269999987992
2.4590919999951666
2.7435370000006287
2.3976989999994203
2.6926019999962136
2.3984480000009967
2.6663370000008513
2.4645100000029174
2.7247960000025273
2.168142999998679
2.7925970000026723
2.1855150000007484
3.1296880000013516
2.4315459999968425
2.4614890000052014
3.073962000001984
3.0965439999945943
3.4393509999972594
3.695821000000876
2.4640950000005546
7.2275149999967425
7.56861299999656
6.143192999999769
5.806074999995303
6.992858000003821
6.403665000000558
6.744534000006297
5.95292199999875
6.290093000004049
6.387706000005267
6.281796000003226
5.803046999993455
3.5615740000025653
4.4276449999998135
2.991688000001602
3.3406900000017004
3.